In [1]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
from DeepSmote import DeepSMOTE
import torch


In [2]:

# Clase personalizada para cargar imágenes desde carpetas
class ViolenceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.load_images()

    def load_images(self):
        violence_dir = os.path.join(self.root_dir, 'violencia')
        no_violence_dir = os.path.join(self.root_dir, 'no_violencia')

        for img_file in os.listdir(violence_dir):
            self.image_paths.append(os.path.join(violence_dir, img_file))
            self.labels.append(1)  # Etiqueta para violencia

        for img_file in os.listdir(no_violence_dir):
            self.image_paths.append(os.path.join(no_violence_dir, img_file))
            self.labels.append(0)  # Etiqueta para no violencia

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label




In [ ]:
def save_synthetic_images(synthetic_images, save_dir, start_index=0):
   
    os.makedirs(save_dir, exist_ok=True)

    # Guardar cada imagen
    for i, img_array in enumerate(synthetic_images):
        # Convertir los datos de imagen a un formato adecuado para guardar
        img_array = (img_array * 255).astype(np.uint8)  # Escalar a rango 0-255
        img = Image.fromarray(np.transpose(img_array, (1, 2, 0)))  # Convertir de tensor a imagen RGB

        # Guardar la imagen con un nombre único
        img.save(os.path.join(save_dir, f"synthetic_image_{start_index + i}.jpg"))

In [4]:
bases=["rlvsd","ubi",]
porcentajes=["1","5","10","25","50","75",]



In [5]:
if __name__ == "__main__":
    torch.cuda.empty_cache()
    print("> Iniciando proceso de Generacion de  muestras con DeepSMOTE")
    for base in bases:
        for i in porcentajes:
            print(">>> Base de Datos: ",base," al ",i,"% de balance")
            rutaDeImagenesDesbalanceadas="./Datasets/Desbalanceadas/"+base+"/Database("+i+")/train/"
            rutaDeImagenesSinteticas="./Datasets/ImagenesSinteticas/"+base+"/Database("+i+")/"
            print("Ruta de imagenes desbalanceadas: "+rutaDeImagenesDesbalanceadas)


            data_dir =  rutaDeImagenesDesbalanceadas # Cambia esto a la ruta de tus carpetas
            batch_size = 8

            # Transformaciones para las imágenes
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ])

            # Cargar dataset de entrenamiento
            dataset = ViolenceDataset(root_dir=data_dir, transform=transform)
            train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

            # Determinar cuántas muestras faltan en la clase "violencia"
            num_violencia = sum([1 for _, label in dataset if label == 1])
            num_no_violencia = sum([1 for _, label in dataset if label == 0])
            samples_to_generate = num_no_violencia - num_violencia
            


            #samples_to_generate=1



            print(f'Muestras actuales de "violencia": {num_violencia}')
            print(f'Muestras actuales de "no violencia": {num_no_violencia}')
            print(f'Muestras a generar: {samples_to_generate}')

            # Inicializar DeepSMOTE
            deep_smote = DeepSMOTE()

            # Entrenar el autoencoder
            deep_smote.train(train_loader)

            # Filtrar el dataset para obtener solo las imágenes de "violencia"
            violence_images = [image for image, label in dataset if label == 1]
            violence_images_tensor = torch.stack(violence_images)

            # Generar muestras sintéticas para la clase minoritaria
            # Generar muestras sintéticas para la clase minoritaria en lotes
            synthetic_images = deep_smote.generate_synthetic_samples_in_batches(violence_images_tensor, samples_to_generate, batch_size=1)

            print("Ruta  de imagenes sinteticas: "+rutaDeImagenesDesbalanceadas)
            # Definir el directorio donde guardar las imágenes
            save_dir = rutaDeImagenesSinteticas  # Cambia esto por el path de tu carpeta de salida

            # Llamar a la función para guardar las imágenes
            save_synthetic_images(synthetic_images, save_dir, start_index=num_violencia)

            print(f"{len(synthetic_images)} imágenes sintéticas guardadas en {save_dir}")

            print("··············································································")
            print()
            print()

> Iniciando proceso de Generacion de  muestras con DeepSMOTE
>>> Base de Datos:  rlvsd  al  1 % de balance
Ruta de imagenes desbalanceadas: ./Datasets/Desbalanceadas/rlvsd/Database(1)/train/
Muestras actuales de "violencia": 59
Muestras actuales de "no violencia": 5936
Muestras a generar: 5877
Epoch [1/500], Loss: 0.1393
Epoch [2/500], Loss: 0.0746
Epoch [3/500], Loss: 0.0586
Epoch [4/500], Loss: 0.0497
Epoch [5/500], Loss: 0.0445
Epoch [6/500], Loss: 0.0398
Epoch [7/500], Loss: 0.0372
Epoch [8/500], Loss: 0.0350
Epoch [9/500], Loss: 0.0331
Epoch [10/500], Loss: 0.0317
Epoch [11/500], Loss: 0.0300
Epoch [12/500], Loss: 0.0284
Epoch [13/500], Loss: 0.0276
Epoch [14/500], Loss: 0.0290
Epoch [15/500], Loss: 0.0253
Epoch [16/500], Loss: 0.0248
Epoch [17/500], Loss: 0.0241
Epoch [18/500], Loss: 0.0236
Epoch [19/500], Loss: 0.0236
Epoch [20/500], Loss: 0.0224
Epoch [21/500], Loss: 0.0219
Epoch [22/500], Loss: 0.0213
Epoch [23/500], Loss: 0.0210
Epoch [24/500], Loss: 0.0205
Epoch [25/500], Lo